In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.random import random

from scipy.integrate import quad
from scipy import constants
from scipy import interpolate

from astropy.cosmology import Planck18
from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.table import Table

from multiprocessing import Pool
import time
###########################
# CONSTANTS
###########################
H0GLOB= 67#69.32#67.9 #69
Om0GLOB=0.319
Xi0Glob =1.
clight = 2.99792458* 10**5#km/s
cosmoglob = Planck18
cosmofast = FlatLambdaCDM(H0=H0GLOB, Om0=Om0GLOB)
H0=cosmoglob.H(0).value
h=H0GLOB/100
#geometrization of masses
Msun=(1.98892)*(10**30)
solarmass_to_m=(constants.G*Msun)/((constants.c)**2)#G/c^2
Mpc_to_m=3.08567758128*(10**22) #this will be used later
#-------miscellanea-----------------------
xmax=100
ymax=100
zmax=100
half_box_size=2

Npoints=1000
Sigma=10
#----------------from distances to redshift----------------------
zz=np.linspace(0,10,50000)
distinterpol=(1+zz)*cosmofast.comoving_distance(zz).value
z_of_dl=interpolate.interp1d(distinterpol,zz)
distinterpol_dc=cosmofast.comoving_distance(zz).value
z_of_dc=interpolate.interp1d(distinterpol_dc,zz)

In [7]:
#------------1/8 of master catalogue------------#
def NewPlacer(redslice):
    one_phi=np.random.choice(temp_phi)
    one_theta=np.random.choice(temp_theta)
    one_zeta=np.random.choice(redslice)
    df_candidates=df
    #Create a list of possible host
    k=1
    #print(df_candidates.empty)
    while True:
        df_candidates=df[(df['ra_gal']>one_phi-k*ang_epsilon) & (df['ra_gal']<one_phi+k*ang_epsilon)]
        df_candidates=df_candidates[(df_candidates['dec_gal']>one_theta-k*ang_epsilon) & (df_candidates['dec_gal']<one_theta+k*ang_epsilon)]
        df_candidates=df_candidates[(df_candidates['true_redshift_gal']>one_zeta-k*red_epsilon) & (df_candidates['true_redshift_gal']<one_zeta+k*red_epsilon)]
        if df_candidates.empty:
            k=k+1 
        if (df_candidates.empty==False):
            break
    #Raul: given a point in space, find nearest N host given the list
    df_candidates=df_candidates.reset_index(drop=True)
    if (df_candidates.shape[0]==1):
        unique_host=df_candidates.head(1)

    if (df_candidates.shape[0]>1):
        parameterdist=np.zeros(df_candidates.shape[0])
        for index, row in df_candidates.iterrows():
            parameterdist[index]=np.sqrt((row['dec_gal']-one_theta)**2+(row['ra_gal']-one_phi)**2+(row['true_redshift_gal']-one_zeta)**2)
        df_candidates['parameterdist']=parameterdist
        df_candidates=df_candidates.sort_values(["parameterdist"], ascending=True) 
        unique_host=df_candidates.head(1)
    #Now, for each value of angles and z, we select a single host
    x_host=unique_host['x_gal'].values[0]/h
    y_host=unique_host['y_gal'].values[0]/h
    z_host=unique_host['z_gal'].values[0]/h
    pp=np.random.uniform(0,2*np.pi)
    costt=np.random.uniform(-1,1)
    tt=np.arccos(costt)
    r=np.random.choice(two_point_radii_cutted)
    #distcut=0.1
    #while True:
    #    r=np.random.choice(two_point_radii)
    #    if(r>=distcut):
    #        break
    xx=r*np.sin(tt)*np.cos(pp)/h
    yy=r*np.sin(tt)*np.sin(pp)/h
    zz=r*np.cos(tt)/h
    xDS=x_host+xx
    yDS=y_host+yy
    zDS=z_host+zz
    Redshift_host=unique_host['true_redshift_gal'].values[0]
    dsdistnoh=np.sqrt(xDS**2+yDS**2+zDS**2)#real comoving distance
    Redshift_DS=z_of_dc(dsdistnoh)
    return(x_host,y_host,z_host,Redshift_host,xDS,yDS,zDS,Redshift_DS)
def spoiler(redshift_positions,number_of_DS):
    Host_x_todf=[]
    Host_y_todf=[]
    Host_z_todf=[]
    Host_redshift_todf=[]
    DS_x_todf=[]
    DS_y_todf=[]
    DS_z_todf=[]
    DS_redshift_todf=[]
    retdf=pd.DataFrame()
    print('I am Working')
    for i in range (len(redshift_positions)-1):
        zmin=redshift_positions[i]
        zmax=redshift_positions[i+1]
        num=int(number_of_DS[i])
        print('i={}'.format(i))
        for k in range(num):
            #if k==0:
            #    print(k)
            #if (k%100==0):
            #    print(k)
            tempred=temp_z_values[(temp_z_values>=zmin)&(temp_z_values<=zmax)]
            flybie=NewPlacer(tempred)
            Host_x_todf=np.append(Host_x_todf,flybie[0])
            Host_y_todf=np.append(Host_y_todf,flybie[1])
            Host_z_todf=np.append(Host_z_todf,flybie[2])
            Host_redshift_todf=np.append(Host_redshift_todf,flybie[3])
            DS_x_todf=np.append(DS_x_todf,flybie[4])
            DS_y_todf=np.append(DS_y_todf,flybie[5])
            DS_z_todf=np.append(DS_z_todf,flybie[6])
            DS_redshift_todf=np.append(DS_redshift_todf,flybie[7])
    data=[Host_x_todf,Host_y_todf,Host_z_todf,Host_redshift_todf,DS_x_todf,DS_y_todf,DS_z_todf,DS_redshift_todf]
    #print(len(Host_x_todf),len(Host_y_todf),len(Host_y_todf),len(Host_redshift_todf),len(DS_x_todf),len(DS_y_todf),len(DS_z_todf),len(DS_redshift_todf))
    mycol=['Host_x','Host_y','Host_z','Host_redshift','DS_x','DS_y','DS_z','DS_redshift']
    for n in range(len(mycol)):
        retdf[mycol[n]]=data[n]
    return retdf

In [8]:
#------------read the flagship-------------------------------------
dat = Table.read('11536.fits', format='fits')
df = dat.to_pandas()#all good, is an only text fits
print(df.columns)
print(df.shape)

Index(['ra_gal', 'dec_gal', 'observed_redshift_gal', 'true_redshift_gal',
       'x_gal', 'y_gal', 'z_gal', 'vx_gal', 'vy_gal', 'vz_gal'],
      dtype='object')
(14047533, 10)


In [9]:
#############################################################################################
###############--------DS-spatial-coordinates-###############################################
#############################################################################################

#Restrict the angular coordinate to be those of the flagship
phi_min=df.ra_gal.min()#*np.pi/180
phi_max=df.ra_gal.max()#*np.pi/180
theta_min=df.dec_gal.min()#*np.pi/180
theta_max=df.dec_gal.max()#*np.pi/180
#Restrict to the right redshift interval
z_min=df.true_redshift_gal.min()
z_max=df.true_redshift_gal.max()
#print('phi_min={}, phi_max={}, \ntheta_min={}, theta_max{}, \nz_min{}, z_max={}'
#      .format(phi_min,phi_max,theta_min,theta_max,z_min,z_max))
#ds_phi=np.random.uniform(0,phi_lim,500000)
ang_epsilon=0.1
red_epsilon=0.01
temp_phi=np.random.uniform(phi_min,phi_max,100000)#degrees
temp_theta=np.random.uniform(theta_min,theta_max,100000)#degrees
temp_z_values=np.loadtxt('redshift_MC.txt')
two_point_radii=np.loadtxt('two_point_radii.txt')
two_point_radii_cutted=np.loadtxt('two_point_radii_cutted.txt')
Numb_DS_of_z_int=np.loadtxt('Numb_DS_of_z_int.txt')
arr_of_int_lim=np.loadtxt('arr_of_int_lim.txt')

In [12]:
start_time = time.time()
with Pool(35) as p:
    EVA00=spoiler(arr_of_int_lim,Numb_DS_of_z_int)
print("--- %s seconds ---" % ((time.time() - start_time)))

I am Working
i=0
i=1
i=2
i=3
i=4
i=5
i=6
i=7
i=8
i=9
i=10
i=11
i=12
i=13
i=14
i=15
i=16
i=17
i=18
i=19
i=20
i=21
i=22
i=23
i=24
i=25
i=26
i=27
i=28
i=29
i=30
i=31
i=32
i=33
i=34
i=35
i=36
i=37
i=38
i=39
i=40
i=41
i=42
i=43
i=44
i=45
i=46
i=47
i=48
i=49
i=50
i=51
i=52
i=53
i=54
i=55
i=56
i=57
i=58
i=59
i=60
i=61
i=62
i=63
i=64
i=65
i=66
i=67
i=68
i=69
i=70
i=71
i=72
i=73
i=74
i=75
i=76
i=77
i=78
i=79
i=80
i=81
i=82
i=83
i=84
i=85
i=86
i=87
i=88
i=89
i=90
i=91
i=92
i=93
i=94
i=95
i=96
i=97
i=98
i=99
i=100
i=101
i=102
i=103
i=104
i=105
i=106
i=107
i=108
i=109
i=110
i=111
i=112
i=113
i=114
i=115
i=116
i=117
i=118
i=119
i=120
i=121
i=122
i=123
i=124
i=125
i=126
i=127
i=128
i=129
i=130
i=131
i=132
i=133
i=134
i=135
i=136
i=137
i=138
i=139
i=140
i=141
i=142
i=143
i=144
i=145
i=146
i=147
i=148
i=149
--- 35846.27286243439 seconds ---
